# 概要

ChainerのMNISTのチュートリアルではいきなり3層モデルで実装しているので、ここでは最も単純な1層モデルを実装します。  
1層モデルは、機械学習の成果物（WeightとBias）が確認しやすいメリットがあります。  
ただ、Chainerだと、

    y = Wx + b

の数式がイメージしにくいです。


# 参考サイト

- [Chainer v4 ビギナー向けチュートリアル](https://qiita.com/mitmul/items/1e35fba085eb07a92560)
- [chainer/examples/mnist at master · chainer/chainer · GitHub](https://github.com/chainer/chainer/tree/master/examples/mnist)
- [ChainerでMNISTの手書き文字認識 - しがないエンジニアのブログ](http://turgure.hatenablog.com/entry/2016/08/04/010219)

In [ ]:
### 共通ライブラリ
# numpy
import numpy as np

# 画像表示
import matplotlib.cm as cm
from matplotlib import pylab as plt

# MNISTデータの取得

[get_mnist関数](https://docs.chainer.org/en/stable/reference/generated/chainer.datasets.get_mnist.html) を使って、Chainerで用意しているMNISTデータセット（70,000件）を取得し、下記のように分割する。

* 訓練データ： 50,000件
* 検証データ： 10,000件
* テストデータ： 10,000件

In [ ]:
from chainer.datasets import mnist, split_dataset_random

### データセットを取得する。すでに取得済みの場合、取得済みデータセットをロードする。
# withlabel=True： 画像データと画像データに対応する数値をタプル形式で取得する
# ndim： 取得する画素配列の次元を指定する。
#    ndim=1： 784 x 1の１次元配列
#    ndim=2： 28 x 28の２次元配列
#    ndim=3：　色 x 28 x 28の３次元配列
#
train_val, test_data = mnist.get_mnist(withlabel=True, ndim=1)
train_data, valid_data = split_dataset_random(train_val, 50000, seed=0)
#print(train_data)
#print(valid_data)
#print(test_data)

## 訓練画像先頭10枚の表示

In [ ]:
print('Training dataset size:', len(train_data))
# 訓練画像先頭10枚の確認
# 2行x5列の画像出力領域を確保
fig, axarr = plt.subplots(2, 5)
# 各出力領域に絵をセットする
for idx in range(10):
    ax = axarr[int(idx / 5)][idx % 5]

    x, t = train_data[idx] 
    ax.imshow(x.reshape(28, 28), cmap = cm.Greys_r)
    ax.set_title(str(t))
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
# 絵を出力する
plt.show()
plt.close()

## 検証画像先頭10枚の表示

In [ ]:
print('Validation dataset size:', len(valid_data))
# validate画像先頭10枚の確認
# 2行x5列の画像出力領域を確保
fig, axarr = plt.subplots(2, 5)
# 各出力領域に絵をセットする
for idx in range(10):
    ax = axarr[int(idx / 5)][idx % 5]

    x, t = valid_data[idx] 
    ax.imshow(x.reshape(28, 28), cmap = cm.Greys_r)
    ax.set_title(str(t))
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
# 絵を出力する
plt.show()
plt.close()

## テスト画像先頭10枚の表示

In [ ]:
print('Test dataset size:', len(valid_data))
# テスト画像先頭10枚の確認
# 2行x5列の画像出力領域を確保
fig, axarr = plt.subplots(2, 5)
# 各出力領域に絵をセットする
for idx in range(10):
    ax = axarr[int(idx / 5)][idx % 5]

    x, t = test_data[idx] 
    ax.imshow(x.reshape(28, 28), cmap = cm.Greys_r)
    ax.set_title(str(t))
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
# 絵を出力する
plt.show()
plt.close()

# Iteratorの作成

ネットワークのパラメータ最適化手法として広く用いられるStochastic Gradient Descent(SGD)という手法では、いくつかのデータを束ねたミニバッチと呼ばれる単位ごとに推論→パラメータ更新を行い、全ミニバッチについて推論→パラメータ更新が完了したことを１epochという単位として、推論時の予測精度が収束するまでepochを繰り返します。  
Chainerで学習データとそのラベルを束ね、ミニバッチに相当するデータセットを作成する機能をIteratorと呼び、ここで使用する[SerialIterator以外にもいくつかのIterator](https://docs.chainer.org/en/stable/reference/iterators.html)が用意されています。

*  [オンライン学習、バッチ学習、ミニバッチ学習の違い](https://ja.stackoverflow.com/questions/48021/%e3%82%aa%e3%83%b3%e3%83%a9%e3%82%a4%e3%83%b3%e5%ad%a6%e7%bf%92-%e3%83%90%e3%83%83%e3%83%81%e5%ad%a6%e7%bf%92-%e3%83%9f%e3%83%8b%e3%83%90%e3%83%83%e3%83%81%e5%ad%a6%e7%bf%92%e3%81%ae%e9%81%95%e3%81%84ß)
* [畳み込みニューラルネ​ットワークの学習にお​けるミニバッチの精度​とは何ですか？](https://jp.mathworks.com/matlabcentral/answers/333915-)

In [ ]:
from chainer import iterators

# ミニバッチのサイズ
# ここでは１２８個のデータを一括りにして、推論→パラメータ更新を行う
batchsize = 128

train_iter = iterators.SerialIterator(train_data, batchsize)
valid_iter = iterators.SerialIterator(valid_data, batchsize, repeat=False, shuffle=False)
test_iter = iterators.SerialIterator(test_data, batchsize, repeat=False, shuffle=False)

# ネットワーク定義

ここでは中間層なしで、入力画像を0〜9の１０クラスに分類するネットワークを作成します。
学習のイメージとしては、下記の数式に対し、各画素配列要素ごとにyを計算し、ラベルと同じかどうかをチェックして、違った場合は同じになるようにW、bを調整していく感じです。  
ちなみに、Wとbの初期値は乱数です（0でも良いらしいのですが、乱数の方が学習速度が早いらしいです）。

    y = Wx + b

In [ ]:
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import Chain

In [ ]:
# ネットワークモデル
# 入力784, 出力10, 単層
class MnistBeginner(Chain):
    def __init__(self):
        super(MnistBeginner, self).__init__()
        with self.init_scope():
            # 第1引数が入力画素配列。784としてしても結果は同じになる。
            self.l1 = L.Linear(None, 10)

    def __call__(self, x):
        return self.l1(x)
    
    def getWeight(self):
        return self.l1.W

    def getBias(self):
        return self.l1.b

net = MnistBeginner()

gpu_id = -1  # CPUを用いる場合は、この値を-1にしてください
if gpu_id >= 0:
    net.to_gpu(gpu_id)

# 最適化手法

* [Chainerで使える最適化手法](https://docs.chainer.org/en/stable/reference/optimizers.html)

In [ ]:
from chainer import optimizers
optimizer = optimizers.SGD(lr=0.01).setup(net)

# 学習ループ

In [ ]:
from chainer.dataset import concat_examples
from chainer.cuda import to_cpu

max_epoch = 20

while train_iter.epoch < max_epoch:

    # ---------- 学習の1イテレーション ----------
    train_batch = train_iter.next()
    x, t = concat_examples(train_batch, gpu_id)

    # 予測値の計算
    y = net(x)

    # ロスの計算
    loss = F.softmax_cross_entropy(y, t)

    # 勾配の計算
    net.cleargrads()
    loss.backward()

    # パラメータの更新
    optimizer.update()
    # --------------- ここまで ----------------

    # 1エポック終了ごとにValidationデータに対する予測精度を測って、
    # モデルの汎化性能が向上していることをチェックしよう
    if train_iter.is_new_epoch:  # 1 epochが終わったら

        # ロスの表示
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.data))), end='')

        valid_losses = []
        valid_accuracies = []
        while True:
            valid_batch = valid_iter.next()
            x_valid, t_valid = concat_examples(valid_batch, gpu_id)

            # Validationデータをforward
            with chainer.using_config('train', False), \
                    chainer.using_config('enable_backprop', False):
                y_valid = net(x_valid)

            # ロスを計算
            loss_valid = F.softmax_cross_entropy(y_valid, t_valid)
            valid_losses.append(to_cpu(loss_valid.array))

            # 精度を計算
            accuracy = F.accuracy(y_valid, t_valid)
            accuracy.to_cpu()
            valid_accuracies.append(accuracy.array)

            if valid_iter.is_new_epoch:
                valid_iter.reset()
                break

        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
            np.mean(valid_losses), np.mean(valid_accuracies)))

# テストデータでの評価
test_accuracies = []
while True:
    test_batch = test_iter.next()
    x_test, t_test = concat_examples(test_batch, gpu_id)

    # テストデータをforward
    with chainer.using_config('train', False), \
            chainer.using_config('enable_backprop', False):
        y_test = net(x_test)

    # 精度を計算
    accuracy = F.accuracy(y_test, t_test)
    accuracy.to_cpu()
    test_accuracies.append(accuracy.array)

    if test_iter.is_new_epoch:
        test_iter.reset()
        break

print('test_accuracy:{:.04f}'.format(np.mean(test_accuracies)))

# Weightの内容

chainerの変数はVariableクラスになっていて、arrayプロパティにnumpy配列が格納されているらしい。

* [chainer.Variable - chainerfan ページ！](https://chainerfan.jimdo.com/%E3%82%AA%E3%83%96%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E3%83%AA%E3%82%B9%E3%83%881/chainer-variable/)

In [ ]:
weights = net.getWeight()
print(weights.shape)

fig, axarr = plt.subplots(2, 5)
for idx in range(10):
    ax = axarr[int(idx / 5)][idx % 5]
    img_src = (weights[idx, :].array * 100).astype(np.int32)

    #print((weights[idx, :].array * 100).astype(np.int32))
    ax.imshow(img_src.reshape(28, 28), cmap = cm.Greys_r)
    ax.set_title(str(idx))
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
plt.show()
plt.close()

# バイアスの表示

In [ ]:
bias = net.getBias()
print(bias.array)